In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt

In [59]:
try:
  cnx = mysql.connector.connect(user='admin', password='qazwsx&*%^',
                              host='shadowfax1.cedjgswziywb.ap-south-1.rds.amazonaws.com',
                              database='shadowfax')
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

In [9]:
db_cursor = cnx.cursor()
# db_cursor.execute('SELECT * FROM products')
df = pd.read_sql('SELECT * FROM products', con=cnx)
df_tags = pd.read_sql('SELECT * FROM tags', con=cnx)
df_tags.set_index('inc_id', inplace=True)
df.set_index('inc_id', inplace=True)
df

,product_name,tags,mrp,image_address,shop_id,ratings,tags_string,quantity
inc_id,,,,,,,,
1,Red lays,"1,2,3,4",20,https://m.media-amazon.com/images/I/7114i49ZZu...,1,5,"fastfood,snacks,chips,grocery",200
2,Blue Lays,"1,2,3,4",20,https://m.media-amazon.com/images/I/71FFI9ErOW...,1,5,"fastfood,snacks,chips,grocery",250
3,Maggi,"1,2,4,5,20",25,https://encrypted-tbn0.gstatic.com/images?q=tb...,1,5,"fastfood, snacks, grocery, noodles, pazzta",100
4,Pazzta,"1,2,4,5,21",35,https://encrypted-tbn0.gstatic.com/images?q=tb...,1,5,"fastfood, snacks, grocery, noodles, maggi",100
5,Protein Bar,"4,6,7,13,2",40,https://www.pngkit.com/png/full/31-319756_whey...,1,5,"protein, chocolate, dairy, grocery, snacks",50
6,Sweet Curd,"8,9,10,13,18",30,https://image.shutterstock.com/image-vector/mi...,2,5,"curd, dessert, sweet, dairy, readytoeat",30
7,Butterscotch-icecream,"9,10,11,13,2",100,https://4.imimg.com/data4/GC/XF/MY-9521221/but...,2,5,"dessert, sweet, cold, dairy, snacks",500
8,Pepsi,"10,11,12,22",45,https://www.pngall.com/wp-content/uploads/2016...,1,5,"beverages, sweet, cold, drinks",400
9,Milk,"6,12,13,22",40,https://w7.pngwing.com/pngs/600/735/png-transp...,1,5,"beverages, protein, dairy, drinks",25


In [8]:
# df_rec1 = df_rec.loc[df_rec['product_name'] == 'Pepsi']
a = list(df['tags_string'])
tag_column = []
for i in range(len(df['tags_string'])):
    prod_tags = a[i].split(',')
    tag_column.append(prod_tags)
# tag_column
df['tags_split'] = tag_column
df

,product_name,tags,mrp,image_address,shop_id,ratings,tags_string,quantity,tags_split
inc_id,,,,,,,,,
1,Red lays,"1,2,3,4",20,https://m.media-amazon.com/images/I/7114i49ZZu...,1,5,"fastfood,snacks,chips,grocery",200,"[fastfood, snacks, chips, grocery]"
2,Blue Lays,"1,2,3,4",20,https://m.media-amazon.com/images/I/71FFI9ErOW...,1,5,"fastfood,snacks,chips,grocery",250,"[fastfood, snacks, chips, grocery]"
3,Maggi,"1,2,4,5,20",25,https://encrypted-tbn0.gstatic.com/images?q=tb...,1,5,"fastfood, snacks, grocery, noodles, pazzta",100,"[fastfood, snacks, grocery, noodles, pazzta]"
4,Pazzta,"1,2,4,5,21",35,https://encrypted-tbn0.gstatic.com/images?q=tb...,1,5,"fastfood, snacks, grocery, noodles, maggi",100,"[fastfood, snacks, grocery, noodles, maggi]"
5,Protein Bar,"4,6,7,13,2",40,https://www.pngkit.com/png/full/31-319756_whey...,1,5,"protein, chocolate, dairy, grocery, snacks",50,"[protein, chocolate, dairy, grocery, snacks]"
6,Sweet Curd,"8,9,10,13,18",30,https://image.shutterstock.com/image-vector/mi...,2,5,"curd, dessert, sweet, dairy, readytoeat",30,"[curd, dessert, sweet, dairy, readytoeat]"
7,Butterscotch-icecream,"9,10,11,13,2",100,https://4.imimg.com/data4/GC/XF/MY-9521221/but...,2,5,"dessert, sweet, cold, dairy, snacks",500,"[dessert, sweet, cold, dairy, snacks]"
8,Pepsi,"10,11,12,22",45,https://www.pngall.com/wp-content/uploads/2016...,1,5,"beverages, sweet, cold, drinks",400,"[beverages, sweet, cold, drinks]"
9,Milk,"6,12,13,22",40,https://w7.pngwing.com/pngs/600/735/png-transp...,1,5,"beverages, protein, dairy, drinks",25,"[beverages, protein, dairy, drinks]"


In [10]:
df_tags

,tag
inc_id,
1,fast food
2,snacks
3,chips
4,grocery
5,noodles
6,protein
7,chocolate
8,curd
9,dessert


SELECT id, category, location
FROM table
WHERE
(
    category LIKE '%keyword%'
    OR location LIKE '%keyword%'
)

In [63]:
db_cursor = cnx.cursor()
# db_cursor.execute('SELECT * FROM products')
tag_input = "fastfood"
query = f"SELECT * FROM shadowfax.products WHERE( tags_string LIKE '%{tag_input}%' );"
df_tags = pd.read_sql(query, con=cnx)
df_tags.set_index('inc_id', inplace=True)
df_tags

,product_name,tags,mrp,image_address,shop_id,ratings,tags_string,quantity
inc_id,,,,,,,,
1,Red lays,"1,2,3,4",20,https://m.media-amazon.com/images/I/7114i49ZZu...,1,5,"fastfood,snacks,chips,grocery",200
2,Blue Lays,"1,2,3,4",20,https://m.media-amazon.com/images/I/71FFI9ErOW...,1,5,"fastfood,snacks,chips,grocery",250
3,Maggi,"1,2,4,5,20",25,https://encrypted-tbn0.gstatic.com/images?q=tb...,1,5,"fastfood, snacks, grocery, noodles, pazzta",100
4,Pazzta,"1,2,4,5,21",35,https://encrypted-tbn0.gstatic.com/images?q=tb...,1,5,"fastfood, snacks, grocery, noodles, maggi",100


In [64]:
df_json = df_tags.to_dict(orient='records')
print(type(df_json[1:-1]))
df_json

<class 'list'>


[{'product_name': 'Red lays',
  'tags': '1,2,3,4',
  'mrp': 20,
  'image_address': 'https://m.media-amazon.com/images/I/7114i49ZZuL._SL1500_.jpg',
  'shop_id': 1,
  'ratings': 5,
  'tags_string': 'fastfood,snacks,chips,grocery',
  'quantity': 200},
 {'product_name': 'Blue Lays',
  'tags': '1,2,3,4',
  'mrp': 20,
  'image_address': 'https://m.media-amazon.com/images/I/71FFI9ErOWL._SX522_.jpg',
  'shop_id': 1,
  'ratings': 5,
  'tags_string': 'fastfood,snacks,chips,grocery',
  'quantity': 250},
 {'product_name': 'Maggi',
  'tags': '1,2,4,5,20',
  'mrp': 25,
  'image_address': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRJXkRPR9CQb-o1AmUXdoEr0irZ_liE6OuPhg&usqp=CAU',
  'shop_id': 1,
  'ratings': 5,
  'tags_string': 'fastfood, snacks, grocery, noodles, pazzta',
  'quantity': 100},
 {'product_name': 'Pazzta',
  'tags': '1,2,4,5,21',
  'mrp': 35,
  'image_address': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR7cQ0eQhqdP2iUqKK4H3D1JbzFVQe4O8wAYw&usqp=CAU',
  'shop_id': 1

In [65]:
final_dict = {}
if df_json == []:
    resp = 404
else:
    resp = 200
final_dict['status'] = resp
final_dict['data'] = df_json
final_dict

{'status': 200,
 'data': [{'product_name': 'Red lays',
   'tags': '1,2,3,4',
   'mrp': 20,
   'image_address': 'https://m.media-amazon.com/images/I/7114i49ZZuL._SL1500_.jpg',
   'shop_id': 1,
   'ratings': 5,
   'tags_string': 'fastfood,snacks,chips,grocery',
   'quantity': 200},
  {'product_name': 'Blue Lays',
   'tags': '1,2,3,4',
   'mrp': 20,
   'image_address': 'https://m.media-amazon.com/images/I/71FFI9ErOWL._SX522_.jpg',
   'shop_id': 1,
   'ratings': 5,
   'tags_string': 'fastfood,snacks,chips,grocery',
   'quantity': 250},
  {'product_name': 'Maggi',
   'tags': '1,2,4,5,20',
   'mrp': 25,
   'image_address': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRJXkRPR9CQb-o1AmUXdoEr0irZ_liE6OuPhg&usqp=CAU',
   'shop_id': 1,
   'ratings': 5,
   'tags_string': 'fastfood, snacks, grocery, noodles, pazzta',
   'quantity': 100},
  {'product_name': 'Pazzta',
   'tags': '1,2,4,5,21',
   'mrp': 35,
   'image_address': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR7cQ0eQhqd